<p style="text-align: center; font-size: 300%"> Computational Finance </p>
<img src="img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 50%;">

# Dealing with Data
## NumPy Arrays
* The most fundamental data type in numerical Python is NumPy's `ndarray`.
* An array is similar to a `list`, except that
  * it can have more than one dimension;
  * its elements are homogenous (they all have the same type, e.g., float).
* NumPy provides a large number of functions (called `ufuncs`) that operate elementwise on arrays. This makes it possible to write *vectorized* code, which allows one to avoid loops (which are slow in Python).

### Constructing Arrays
* Arrays can be constructed using the `array` function which takes sequences (e.g, lists), and converts them into arrays. The data type is inferred automatically or can be specified.

In [73]:
import numpy as np
a=np.array([1, 2, 3, 4])
a.dtype

dtype('int64')

In [74]:
a=np.array([1, 2, 3, 4],dtype='float64') #or np.array([1., 2., 3., 4.])
a.dtype

dtype('float64')

* Python uses C++ data types which differ from Python (though `float64` is equivalent to Python's `float`).

* Nested lists result in multidimensional arrays. We won't need anything beyond two-dimensional (i.e., a matrix or table).

In [98]:
b=np.array([[1., 2.], [3., 4.]])
print(b)

[[ 1.  2.]
 [ 3.  4.]]


In [99]:
b.ndim #Number of dimensions

2

In [102]:
b.shape #number of rows and columns

(2, 2)

* Indexing and slicing operations are similar to lists:

In [105]:
b[0,0] #indexing [row, column]. Equivalent to b[0][0]

1.0

In [116]:
b[:,0] #First column. Note that this yields a 1-dimensional array, not a matrix  

array([ 1.,  3.])

## The `pandas` library
### Dataframes


### Fetching Data
* `pandas_datareader` is a package that makes it easy to fetch financial data from the web.
* It used to be included in pandas (and therefore Anaconda). In newer versions, you'll have to do `conda install pandas-datareader` to install it
* See http://pandas-datareader.readthedocs.io/en/latest/remote_data.html for the documentation.

In [18]:
#!conda install -y pandas-datareader #uncomment to install. (Note ! executes shell commands)
import pandas_datareader.data as web

In [38]:
start = pd.datetime(2010, 1, 1)
end = pd.datetime(2013, 1, 27)
f = web.DataReader("AAPL", 'yahoo', start, end)


## Plotting with `matplotlib`